In [1]:
import warnings, re
warnings.filterwarnings("ignore")
from ipywidgets import  Image, VBox, Label, HBox, FileUpload
from keras.models import Sequential, Model, load_model
from keras.preprocessing import image
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [2]:
input_shape = (720,720,3)
cls = load_model("inception_based.h5")
fltr = re.compile("C[A-Z]Y")

In [3]:
data = pd.read_csv('test_names.csv')
data.columns=['filename']
data['label'] = data.applymap(lambda x: 1 if len(fltr.findall(x)) > 0 else 0)

In [4]:
def get_image(img_file, dir_name='Test'):
    with open('D:/OGRDS/Image Classification/{}/'.format(dir_name)+img_file, 'rb') as f:
        return f.read()

def test_iterator(data,dir_name='Test', batch_size=32):
    train_files = data.values
    i = 0
    features = []
    for file in train_files:
        yield preprocess_image(file[0], dir_name)
    
def preprocess_image(img_file, dir_name='Test'):
    img = image.load_img('D:/OGRDS/Image Classification/{}/'.format(dir_name)+img_file, target_size=input_shape)
    img = image.img_to_array(img)
    img = img/255
    return np.array([img])

def display_images(upc):
    test_data = data[data.filename.str.contains(upc)]
    test_labels = cls.predict_generator(test_iterator(test_data,'Test'), steps=len(test_data))
    test_data['prediction'] = list(map(np.argmax,test_labels))

    grid = HBox()
    boxes = []
    for row in test_data.values:
        img = HBox([Image(value=get_image(row[0], 'Test'), width=100,height=100)])
        if row[2] == 1:
            img.layout.justify_content = 'space-around'
            img.layout.border = '3px solid green'
        with open('correct.png' if row[1] == row[2] else 'wrong.png', 'rb') as f:
            R_W = Image(value=f.read(), format='png', width=20,height=20)

        boxes.append(VBox([img,R_W]))


    grid.children = boxes
    grid.layout.flex_flow = 'row wrap'
    grid.layout.justify_content = 'space-between'
    grid.layout.align_content = 'center'
    grid.layout.width = '80%'
    display(grid)

In [ ]:
display_images('0070942123518')

In [4]:
a = FileUpload(multiple = True)

In [5]:
a

FileUpload(value={}, description='Upload', multiple=True)

In [41]:
a.metadata

[{'lastModified': 1559246462000,
  'name': 'AE_0070942123518_NPPFM_C100_C_20190501171328554.jpg',
  'size': 184104,
  'type': 'image/jpeg'},
 {'lastModified': 1559246462000,
  'name': 'AE_0070942123518_NPPFM_C100_C_20190501171334592.jpg',
  'size': 169056,
  'type': 'image/jpeg'},
 {'lastModified': 1559246496000,
  'name': 'AE_3073781011456_NPPFM_C100_C_20180217160028778.jpg',
  'size': 125082,
  'type': 'image/jpeg'},
 {'lastModified': 1559246496000,
  'name': 'AE_3073781011456_NPPFM_C100_C_20180217160029592.jpg',
  'size': 130051,
  'type': 'image/jpeg'}]

In [27]:
grid = VBox()

In [38]:
grid.children = [Image(value=x, width=100, height=100) for x in a.data]


In [40]:
grid.layout.flex_flow = 'row wrap'
grid.layout.justify_content = 'space-between'
grid.layout.align_content = 'center'
grid.layout.width = '80%'
grid